In [0]:
%cd /content/drive/My Drive/Colab Notebooks/DS_Lab/Data/

/content/drive/My Drive/Colab Notebooks/DS_Lab/Data


In [0]:
###Doc du lieu:
import csv
with open('death_rate.csv', 'w') as fileCSV:
  writer = csv.writer(fileCSV, delimiter = ',', quotechar = '"', quoting = csv.QUOTE_MINIMAL)

  with open('death_rate.txt', 'r') as fileTXT:
    for line in fileTXT:
      d_line = line.split()
      writer.writerow(d_line)
  fileTXT.close()
fileCSV.close()


In [0]:
import pandas as pd
import numpy as np
data = pd.read_csv('death_rate.csv')

X = data.drop(columns=['Index', 'Death_rate'])
y = data['Death_rate']
X = np.array(X)
y = np.array(y)
print(X.shape)

(60, 15)


In [0]:
##Chuan hoa du lieu:
X_min = X.min(axis = 0)
X_max = X.max(axis = 0)
X_delta = X_max - X_min
X = (X-X_min)/X_delta
X = np.c_[np.ones((X.shape[0], 1)), X]

In [0]:
###Trien khai mo hinh:
class RidgeRegression:
  def __init__(self):
    return

  #FIT voi phep nhan ma tran:
  def fit(self, X_train, y_train, LAMBDA):
    w = np.linalg.pinv(X_train.T.dot(X_train) + LAMBDA*np.identity(X_train.shape[1])).dot(X_train.T).dot(y_train)
    return w

  #FIT voi GD
  def fit_GD(self, X_train, y_train, LAMBDA, learning_rate, batch_size=20, epoch=50):
    w = np.random.randn(X_train.shape[1])
    last_loss = 10e8
  
    for i in range(epoch):

      #xao tron
      ids = range(0, X.shape[0])
      ids = np.random.shuffle(ids)
      X_train = X_train[ids]
      y_train = y_train[ids]

      #chia ra cac batch va huan luyen
      numOfBatch = X_train.shape[0]/batch_size
      for ib in numOfBatch:
        index = ib*batch_size
        Xb = X_train[index: index + batch_size]
        yb = y_train[index: index + batch_size]
        grad = Xb.T.dot(X.dot(w)-yb) + LAMBDA*w
        w -= learning_rate*grad

      new_loss = self.LOSS(self.predict(X_train, w), y_train)
      if np.abs(new_loss - last_loss) < 1e-5:
        break
      last_loss = new_loss

    return w

  #Tinh ham loss
  def LOSS(self, y_pred, y_real):
    return 1./(y_real.shape[0])*np.linalg.norm(y_pred - y_real)**2
  
  #Du doan ket qua:
  def predict(self, X_pred, w):
    return X_pred.dot(w)
  #Cross-validation
  def cross_validation(self, X_train, y_train, num_folds, LAMBDA):
    #xao tron bo du lieu:
    ids = np.array(range(X_train.shape[0]))
    np.random.shuffle(ids) #cai nay tra ve None nha :)))) 17/4/20
    X_train = X_train[ids]
    y_train = y_train[ids]
    
    n = X_train.shape[0]
    
    #chia thanh cac fold:
    odd = n % num_folds
    num_each = n / num_folds
    valid_ids = np.split(ids[:(n - odd)], num_folds)
    valid_ids[-1] = np.append(valid_ids[-1], ids[n - odd:])
    train_ids = [[k for k in ids if k not in valid_ids[i]] for i in range(num_folds)]

    #tim AvG_losss:
    avg_loss = 0
    for i in range(num_folds):
      train_X = X_train[train_ids[i]]
      train_y = y_train[train_ids[i]]
      valid_X = X_train[valid_ids[i]]
      valid_y = y_train[valid_ids[i]]
      w = self.fit(train_X, train_y, LAMBDA)
      y_pred = self.predict(valid_X, w)
      loss = self.LOSS(y_pred, valid_y)
      avg_loss += loss
    return avg_loss/num_folds

  #tim lambda tot nhat trong 1 mien cho truoc, tra ve gia tri lambda tot nhat va gia tri loss t/ung:
  def range_scan(self, L_values, min_loss, X_train, y_train):
    for L in L_values:
      loss = self.cross_validation(X_train = X_train, y_train = y_train, 
                                   num_folds=5, LAMBDA = L)
      if loss < min_loss:
        min_loss = loss
        best_L = L
    return min_loss, best_L
  
  #Tim lambda tot nhat:
  def get_best_lambda(self, X_train, y_train):
    L_values = range(50) #khoi tao bo gia tri lambda ban dau
    min_loss, best_L = self.range_scan(L_values, min_loss = 10e8, X_train = X_train, y_train = y_train) 
       #buoc nay tim best_L trong 1 mien cho truoc, sau do tim best_L trong khoang hep hon tu gia tri vua tim duoc
    C = range(max(0, best_L-1)*1000, (best_L+1)*1000, 1)
    L_values = [k*1./1000 for k in C]

    #tim best_L chinh xac hon: 
    min_loss, best_L = model.range_scan(L_values, min_loss, X_train, y_train)

    return best_L


In [0]:
#Trien khai:

if __name__ == '__main__':
  #Lay du lieu va chuan hoa <da lam>
  X_train = X[:50]
  y_train = y[:50]
  X_test = X[50:]
  y_test = y[50:]

  model = RidgeRegression()
  #Tim lambda tot nhat:
  LAMBDA = model.get_best_lambda(X_train = X_train, y_train = y_train)
  print("The best LAMBDA: ", LAMBDA)
  
  #Huan luyen mo hinh:
  w = model.fit(X_train, y_train, LAMBDA)

  #Du doan tren tap test:
  y_pred = model.predict(X_test, w)

  #Loss thu duocj:
  print("Loss: ", model.LOSS(y_test, y_pred))

The best LAMBDA:  0.025
Loss:  1412.7984615709365


In [0]:
#Thu nghiem lai voi thu vien
from sklearn.linear_model import LinearRegression

model1 = LinearRegression()
model1.fit(X_train, y_train)
y_pred = model1.predict(X_test)

N = y_pred.shape[0]
print(1./N * np.linalg.norm(y_pred-y_test)**2)
print(X_train[0])

1584.5425212262771
[1.         0.52       0.21126761 0.36363636 0.40322581 0.68852459
 0.72727273 0.61506276 0.21821264 0.21220159 0.33976834 0.13529412
 0.0309119  0.04402516 0.20938628 0.6       ]


In [0]:
a = [(1, 2), (3, 4), (5, 6)]
b = dict(a)
b

{1: 2, 3: 4, 5: 6}